In [1]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

In [2]:
import shutil

shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [1]:
#import sys
#sys.path.insert(0,'./')
# Install pycocotools
# !git clone https://github.com/cocodataset/cocoapi.git
# !ls
# %cd cocoapi/PythonAPI
# !ls
# !pip install cython
# !pip install numpy
# !python setup.py build_ext install

fatal: destination path 'cocoapi' already exists and is not an empty directory.
PyTorch-object-detection-1.ipynb cocoapi
build                            config.json
/Users/gkv/Code/PT-Azure-maskrcnn/cocoapi/PythonAPI
Makefile             dist                 setup.py
PennFudanPed         engine.py            test.zip
__pycache__          pycocoDemo.ipynb     transforms.py
build                pycocoEvalDemo.ipynb utils.py
coco_eval.py         pycocotools          vision
coco_utils.py        pycocotools.egg-info
cocoapi              script.py
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
reading manifest file 'pycocotools.egg-info/SOURCES.txt'
writing manifest file 'pycocotools.e

In [3]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


In [4]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [6]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-02-16T23:53:03.831000+00:00', 'errors': None, 'creationTime': '2020-02-05T18:19:06.976503+00:00', 'modifiedTime': '2020-02-05T18:20:28.120151+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}


In [8]:
import urllib.request
 
url = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
 
urllib.request.urlretrieve(url, './test.zip')

('./test.zip', <http.client.HTTPMessage at 0x10cdfdaf0>)

In [9]:
from zipfile import ZipFile

zip = ZipFile(file='./test.zip')
zip.extractall()

In [10]:
!ls PennFudanPed/

Annotation            PedMasks              readme.txt
PNGImages             added-object-list.txt


In [11]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d


In [12]:
ds.upload('./PennFudanPed', target_path='data', overwrite=False)

Uploading an estimated of 512 files
Target already exists. Skipping upload for data/added-object-list.txt
Target already exists. Skipping upload for data/readme.txt
Target already exists. Skipping upload for data/PedMasks/PennPed00048_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00049_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00072_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00073_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00001_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00093_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00092_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00076_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00077_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00005_mask.png
Target already exists. Skipping upload for

Target already exists. Skipping upload for data/PedMasks/FudanPed00059_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00062_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00063_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00032_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00033_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00008_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00009_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00041_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00040_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00036_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00037_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00045_mask.png
Target already exists. Skipping upload for data/

Target already exists. Skipping upload for data/Annotation/FudanPed00063.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00062.txt
Target already exists. Skipping upload for data/Annotation/PennPed00059.txt
Target already exists. Skipping upload for data/Annotation/PennPed00065.txt
Target already exists. Skipping upload for data/Annotation/PennPed00071.txt
Target already exists. Skipping upload for data/Annotation/PennPed00067.txt
Target already exists. Skipping upload for data/Annotation/PennPed00073.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00074.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00060.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00048.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00049.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00061.txt
Target already exists. Skipping upload for data/Annotation/PennPed00072.txt
Targe

Target already exists. Skipping upload for data/Annotation/PennPed00008.txt
Target already exists. Skipping upload for data/Annotation/PennPed00020.txt
Target already exists. Skipping upload for data/Annotation/PennPed00034.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00033.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00027.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00026.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00032.txt
Target already exists. Skipping upload for data/Annotation/PennPed00035.txt
Target already exists. Skipping upload for data/Annotation/PennPed00021.txt
Target already exists. Skipping upload for data/Annotation/PennPed00009.txt
Target already exists. Skipping upload for data/Annotation/PennPed00037.txt
Target already exists. Skipping upload for data/Annotation/PennPed00023.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00018.txt
Target 

Target already exists. Skipping upload for data/PNGImages/PennPed00024.png
Target already exists. Skipping upload for data/PNGImages/PennPed00030.png
Target already exists. Skipping upload for data/PNGImages/PennPed00031.png
Target already exists. Skipping upload for data/PNGImages/PennPed00025.png
Target already exists. Skipping upload for data/PNGImages/PennPed00019.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00022.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00036.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00032.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00026.png
Target already exists. Skipping upload for data/PNGImages/PennPed00021.png
Target already exists. Skipping upload for data/PNGImages/PennPed00035.png
Target already exists. Skipping upload for data/PNGImages/PennPed00009.png
Target already exists. Skipping upload for data/PNGImages/PennPed00008.png
Target already exists

$AZUREML_DATAREFERENCE_26d2302af8f64b43a3c0bc9a249bffab

In [13]:
from azureml.core import Dataset
# create a FileDataset pointing to files in 'animals' folder and its subfolders recursively
datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "1cd2dd50-ae6d-44f9-81e2-f0044c862efe",
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

In [13]:
# dataset_name = 'penn_ds'

# # Get a dataset by name
# penn1_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [17]:
# from PIL import Image
# import os
# import glob


# with penn_ds.mount() as mount_context:
#     # list top level mounted files and folders in the dataset
#     imgs_path =(os.path.join(mount_context.mount_point, 'PNGImages'))
#     print(f'imgs_path is {imgs_path}')
#     print(list(sorted(os.listdir(imgs_path))))
#     Image.open(os.path.join(mount_context.mount_point, 'PNGImages/PennPed00036.png'))
penn1_ds.as_named_input('test').as_mount()

In [15]:
# PennFudanDataset(penn_ds)

In [22]:
!ls
%cd ..
!ls

LICENSE     README.rst  hubconf.py  setup.cfg   test        tox.ini
MANIFEST.in docs        references  setup.py    torchvision
/Users/gkv/MS/pytorch-object
PennFudanPed                     pytorch-peds
PyTorch-object-detection-1.ipynb script.py
coco_eval.py                     test.zip
coco_utils.py                    transforms.py
config.json                      utils.py
engine.py                        vision


In [24]:
!git clone https://github.com/pytorch/vision.git

!git checkout v0.3.0

%cd vision
!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../
%cd ..

fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git
/Users/gkv/MS/pytorch-object/vision
/Users/gkv/MS/pytorch-object


In [26]:
import shutil

os.listdir()

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./vision/references/detection/'+ file + '.py', project_folder)


In [27]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

In [37]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile_contents_of_your_base_image=f.read()
my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 

In [42]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

# follow pattern from here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-environments-for-training

# Add training script to run config
runconfig = ScriptRunConfig(source_directory=".", script="script.py")

# Attach compute target to run config
runconfig.run_config.target = "local"

my_env.docker.base_image = None
# Attach environment to run config
runconfig.run_config.environment = my_env

# Submit run 
run = experiment.submit(runconfig)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1582496081_2fd369c5,
Type: azureml.scriptrun,
Status: Starting)


In [43]:
# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1582496081_2fd369c5', 'target': 'local', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'local', 'ContentSnapshotId': 'be542fea-e9cc-4a28-8f2f-c93815655bb3', 'azureml.git.repository_uri': 'https://github.com/gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'https://github.com/gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': '618dbb834d141b9c9dda355c158c52ea77e45dc6', 'mlflow.source.git.commit': '618dbb834d141b9c9dda355c158c52ea77e45dc6', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': [], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'local', 'dataReferences': {}, 'data': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': 'maskr-docker', 'version': 'Autosave_2020-02-23T22:11:06Z_168aa0dc'

In [44]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582496081_2fd369c5
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582496081_2fd369c5?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 38906
Running: ['/bin/bash', '/private/var/folders/zq/sdr9gtc92pd15gf53xb1tct80000gn/T/azureml_runs/pytorch-peds_1582496081_2fd369c5/azureml-environment-setup/docker_env_checker.sh']

Materialized image not found on target: azureml/azureml_314fdf099d7c5d83ce53da44e576a098


Logging experiment preparation status in history service.
Running: ['/bin/bash', '/private/var/folders/zq/sdr9gtc92pd15gf53xb1tct80000gn/T/azureml_runs/pytorch-peds_1582496081_2fd369c5/azureml-environment-setup/docker_env_builder.sh']
Running: ['docker', 'build', '-f', 'azureml-environment-setup/Dockerfile', '-

Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libkrb5support0 amd64 1.16-2ubuntu0.1 [30.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libk5crypto3 amd64 1.16-2ubuntu0.1 [85.6 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic/main amd64 libkeyutils1 amd64 1.5.9-9.2ubuntu2 [8720 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libkrb5-3 amd64 1.16-2ubuntu0.1 [279 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgssapi-krb5-2 amd64 1.16-2ubuntu0.1 [122 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libssl1.0.0 amd64 1.0.2n-1ubuntu5.3 [1088 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 openssh-client amd64 1:7.6p1-4ubuntu0.3 [614 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnl-3-200 amd64 3.2.29-0ubuntu3 [52.8 kB]
Get:15 http://archive.ubuntu.com/ubuntu bionic/main amd64 libnl-route-3-200 amd64 3.2.29-0ubuntu3 [146 kB]
Get:16 http://arch

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Building docker image failed with exit code: 100",
        "details": []
    },
    "time": "2020-02-23T22:29:46.207038Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Building docker image failed with exit code: 100\",\n        \"details\": []\n    },\n    \"time\": \"2020-02-23T22:29:46.207038Z\"\n}"
    }
}

In [26]:
from azureml.train.dnn import PyTorch

script_params = {
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='script.py',
                    use_gpu=True,
                    pip_packages=['torchvision>=0.5.0'])

WARNING - framework_version is not specified, defaulting to version 1.3.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['torchvision']. We cannot guarantee image build will succeed.


In [14]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        with self.dataset.mount() as mount_context:
            self.imgs = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        with self.dataset.mount() as mount_context:
            img_path = os.path.join(mount_context.mount_point, "PNGImages", self.imgs[idx])
            mask_path = os.path.join(mount_context.mount_point, "PedMasks", self.masks[idx])
            img = Image.open(img_path).convert("RGB")
            # note that we haven't converted the mask to RGB,
            # because each color corresponds to a different instance
            # with 0 being background
            mask = Image.open(mask_path)

            mask = np.array(mask)
            # instances are encoded as different colors
            obj_ids = np.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]

            # split the color-encoded mask into a set
            # of binary masks
            masks = mask == obj_ids[:, None, None]

            # get bounding box coordinates for each mask
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(masks[i])
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            masks = torch.as_tensor(masks, dtype=torch.uint8)

            image_id = torch.tensor([idx])
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd

            if self.transforms is not None:
                img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [16]:
!pip install --upgrade torchvision

Requirement already up-to-date: torchvision in /Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages (0.5.0)


In [27]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [19]:
import sys
sys.path.insert(0,'./')

In [20]:
from platform import python_version

print(python_version())
help("modules")

3.8.1

Please wait a moment while I gather a list of all available modules...



/Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."
/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


Cython              asynchat            inspect             random
IPython             asyncio             io                  re
OpenSSL             asyncore            ipaddress           readline
PIL                 atexit              ipykernel           reprlib
PyQt5               attr                ipykernel_launcher  requests
__future__          audioop             ipython_genutils    requests_oauthlib
_abc                automl              ipywidgets          resource
_ast                autoreload          isodate             rlcompleter
_asyncio            azureml             itertools           rmagic
_bisect             backcall            jaraco              runpy
_blake2             backports           jedi                sched
_bootlocale         base64              jeepney             secrets
_bz2                bdb                 jinja2              secretstorage
_cffi_backend       binascii            jmespath            select
_codecs             binhex           

/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  __import__(info.name)


In [28]:
# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T

# def get_transform(train):
#     transforms = []
#     # converts the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during training, randomly flip the training images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)


In [33]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset(penn_ds, get_transform(train=True))
# dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# # define training and validation data loaders
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn)

# data_loader_test = torch.utils.data.DataLoader(
#     dataset_test, batch_size=1, shuffle=False, num_workers=4,
#     collate_fn=utils.collate_fn)

In [34]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # our dataset has two classes only - background and person
# num_classes = 2

# # get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
# # move model to the right device
# model.to(device)

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# # and a learning rate scheduler which decreases the learning rate by
# # 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [1]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

NameError: name 'train_one_epoch' is not defined

In [ ]:
# # pick one image from the test set
# img, _ = dataset_test[0]
# # put the model in evaluation mode
# model.eval()
# with torch.no_grad():
#     prediction = model([img.to(device)])

In [ ]:
# Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())